In [ ]:
### used at DICOM only!!!

In [1]:
import os
import monai
import torch
from monai.data import DataLoader, Dataset, load_decathlon_datalist, NibabelWriter
from monai.data.utils import no_collation
from monai.transforms import (
    Compose,
    EnsureChannelFirstd,
    EnsureTyped,
    LoadImaged,
    Orientationd,
    Spacingd,
)

In [2]:
monai.config.print_config()

MONAI version: 1.2.0
Numpy version: 1.24.4
Pytorch version: 2.0.1+cu118
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: /usr/local/lib/python3.8/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.3.0
Nibabel version: 5.1.0
scikit-image version: 0.21.0
Pillow version: 10.0.0
Tensorboard version: 2.14.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.2+cu118
tqdm version: 4.66.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 2.0.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: 2.6.0
pynrrd version: 1.0.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [3]:
spacing = [0.703125, 0.703125, 1.25]
# data_base_dir = '/data/HC_Images_resample/'
# data_base_dir = '/data/MSD_Images_resample/'
# data_base_dir = '/data/MSD_Masks_resample/'
data_base_dir = '/data/LUNA16_Images_resample2/'
# data_list_file_path = '/data/output/hc_train_val2.json'
# data_list_file_path = '/data/output/hc_test2.json'
# data_list_file_path = '/data/output/msd_train_val2.json'
# data_list_file_path = '/data/output/msd_test2.json'
# data_list_file_path = '/data/output/msd_test2.json'
data_list_file_path = '/data/output/LUNA16_datasplit/mhd_original/dataset_fold0.json'
# orig_data_base_dir = '/data/HC/'
# orig_data_base_dir = '/data/MSD-exams/'
# orig_data_base_dir = '/data/MSD-masks/'
orig_data_base_dir = '/data/LUNA16_Images/'

In [4]:
process_transforms = Compose(
    [
        LoadImaged(
            keys=["image"],
            meta_key_postfix="meta_dict",
            reader="itkreader",
            affine_lps_to_ras=True
        ),
        EnsureChannelFirstd(keys=["image"]),
        EnsureTyped(keys=["image"], dtype=torch.float16),
        Orientationd(keys=["image"], axcodes="RAS"),
        Spacingd(keys=["image"], pixdim=spacing, padding_mode="border"),
    ]
)

monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.


In [5]:
for data_list_key in ["training", "validation"]:
# for data_list_key in ["test"]:
    process_data = load_decathlon_datalist(
        data_list_file_path,
        is_segmentation=True,
        data_list_key=data_list_key,
        base_dir=orig_data_base_dir
    )
    process_ds = Dataset(
        data=process_data,
        transform=process_transforms
    )
    process_loader = DataLoader(
        process_ds,
        batch_size=1,
        shuffle=False,
        pin_memory=False,
        collate_fn=no_collation
    )

    print("-" * 10)
    for batch_data in process_loader:
        for batch_data_i in batch_data:
            print(batch_data_i["image_meta_dict"]["filename_or_obj"])
            print(batch_data_i["image"].meta)
            # subj_id = batch_data_i["image_meta_dict"]["filename_or_obj"].split("/")[3] + '.nii.gz'
            # subj_id = batch_data_i["image_meta_dict"]["filename_or_obj"].split("/")[3]
            # subj_id = batch_data_i["image_meta_dict"]["filename_or_obj"].split("/")[-1]
            # new_filename = os.path.join(data_base_dir, subj_id)
            # writer = NibabelWriter()
            # writer.set_data_array(data_array=batch_data_i["image"])
            # writer.set_metadata(meta_dict=batch_data_i["image"].meta)
            # writer.write(new_filename, verbose=True)

----------
/data/LUNA16_Images/1.3.6.1.4.1.14519.5.2.1.6279.6001.250397690690072950000431855143.mhd
{'spacing': array([0.76171899, 0.76171899, 1.25      ]), original_affine: array([[  -0.76171899,    0.        ,    0.        ,  176.800003  ],
       [   0.        ,   -0.76171899,    0.        ,  138.5       ],
       [   0.        ,    0.        ,    1.25      , -315.25      ],
       [   0.        ,    0.        ,    0.        ,    1.        ]]), space: RAS, affine: tensor([[   0.7031,    0.0000,    0.0000, -212.4384],
        [   0.0000,    0.7031,    0.0000, -250.7384],
        [   0.0000,    0.0000,    1.2500, -315.2500],
        [   0.0000,    0.0000,    0.0000,    1.0000]], dtype=torch.float64), spatial_shape: array([512, 512, 246]), original_channel_dim: nan, 'filename_or_obj': '/data/LUNA16_Images/1.3.6.1.4.1.14519.5.2.1.6279.6001.250397690690072950000431855143.mhd'}
/data/LUNA16_Images/1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886.mhd
{'spacing': array([0.82

KeyboardInterrupt: 